In [1]:
from random import shuffle
import glob,h5py, cv2, scipy, pickle, itertools, re, gc
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm

filename = 'Datasets/'+'final_imit_RD-RA2_128.pkl'
print('File to be created: ' + filename)

File to be created: Datasets/final_imit_RD-RA2_128.pkl


In [2]:
# load videos
subjects = ['10 dec emre mahbub', '11 jan mahbub emre ladi', '12 jan ozgur mahbub ladi emre', 
            '13 jan ridvan ademola sean akthar emre', '14 jan ashwanth zeki', '15 apr ozgur',
           '15 jan emin', '19 apr sean', '19 apr sevgi', '21 apr emin', '22 apr ladi', '22 apr sean',
           '23 apr alperen', '23 apr mahbub', '5 apr sevgi']
# subjects = ['6 apr blake', '15 apr darrin', '16 apr devon asl2', '20 apr caroline', '21 apr blake']
mainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/'
RD_files = []
for i in range(len(subjects)):
    addr = glob.glob(mainpath+subjects[i]+'/rangeDoppler/'+'*.avi')
    RD_files.append(addr)
RD_files = sum(RD_files, [])
print('Num. of label files: '+str(len(RD_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+RD_files[i])

Num. of label files: 1320
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDoppler/11050014_1607634027_2.avi
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDoppler/11010014_1607632426_2.avi
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDoppler/11050014_1607635095_2.avi
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDoppler/11040014_1607633760_5.avi
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDoppler/11030014_1607634656_1.avi


In [3]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [4]:
labels = []
common = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/common labels/rangeDoppler/'
cnt = []
for i in tqdm(range(len(RD_files))):
    seq = RD_files[i][-22]
    it = RD_files[i][-5]
    
    if seq == '1' and it == '1':
        labels.append(np.squeeze(load_file(common+'1_1.txt')))
        cnt.append(0)
    elif seq == '1' and it != '1':
        labels.append(np.squeeze(load_file(common+'1_2to5.txt')))
        cnt.append(1)
    elif seq == '2' and it == '1':
        labels.append(np.squeeze(load_file(common+'2_1.txt')))
        cnt.append(2)
    elif seq == '2' and it != '1':
        labels.append(np.squeeze(load_file(common+'2_2to5.txt')))
        cnt.append(3)
    elif seq == '3' and it == '1':
        labels.append(np.squeeze(load_file(common+'3_1.txt')))
        cnt.append(4)
    elif seq == '3' and it != '1':
        labels.append(np.squeeze(load_file(common+'3_2to5.txt')))
        cnt.append(5)
    elif seq == '4' and it == '1':
        labels.append(np.squeeze(load_file(common+'4_1.txt')))
        cnt.append(6)
    elif seq == '4' and it != '1':
        labels.append(np.squeeze(load_file(common+'4_2to5.txt')))
        cnt.append(7)
    elif seq == '5' and it == '1':
        labels.append(np.squeeze(load_file(common+'5_1.txt')))
        cnt.append(8)
    elif seq == '5' and it != '1':
        labels.append(np.squeeze(load_file(common+'5_2to5.txt')))
        cnt.append(9)
labels = np.array(labels)
print(labels.shape)
print(Counter(cnt))

100%|██████████| 1320/1320 [00:57<00:00, 23.01it/s]

(1320, 605)
Counter({9: 212, 1: 212, 7: 212, 3: 212, 5: 208, 6: 53, 2: 53, 8: 53, 0: 53, 4: 52})


In [5]:
RA_files = []
for i in range(len(RD_files)):
    fname = RD_files[i].replace('rangeDoppler','rangeDOA')
    RA_files.append(fname)
    
print('Num. of label files: '+str(len(RA_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+RA_files[i])

Num. of label files: 1320
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDOA/11050014_1607634027_2.avi
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDOA/11010014_1607632426_2.avi
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDOA/11050014_1607635095_2.avi
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDOA/11040014_1607633760_5.avi
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/rangeDOA/11030014_1607634656_1.avi


In [6]:
train_RD, test_RD, y_train, y_test, train_RA, test_RA = train_test_split(RD_files, labels, RA_files, test_size=0.2, random_state=1)

In [7]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [8]:
def pad_video(video):
    if video.shape[0] < num_frames:
        cnt = 1
        numpad = num_frames-video.shape[0]
        video_padded = np.concatenate((video, np.zeros((numpad,video.shape[1],video.shape[2],video.shape[3]))), axis = 0)
#         while video_padded.shape[0] < num_frames:
#             video_padded = np.concatenate((video, video[-(num_frames-len(video)):]), axis = 0)
#             print('Padding '+str(cnt)+'/'+str(num_frames-video.shape[0]))
#             cnt += 1
    else:
        video_padded = video[:num_frames]
#     video_padded = video[:-1]
    return video_padded

In [9]:
width, height, channels = 128, 128, 3 
num_frames = 605

In [10]:
data = []
data.append(y_train)
data.append(y_test)
del labels, RA_files, RD_files, y_train, y_test
gc.collect()

120

In [11]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 80
__builtins__ 80
_ih 192
_oh 240
_dh 72
In 192
Out 240
get_ipython 64
exit 56
quit 56
_ 28
__ 49
___ 49
_i 160
_ii 104
_iii 643
_i1 469
shuffle 64
glob 80
h5py 80
cv2 80
scipy 80
pickle 80
itertools 80
re 80
gc 80
np 80
clear_output 136
read_csv 136
to_categorical 136
train_test_split 136
Counter 1056
tqdm 2000
filename 83
_i2 865
subjects 184
mainpath 104
i 28
addr 296
_i3 220
load_file 136
_i4 1536
common 131
cnt 11520
seq 58
it 58
_i5 296
fname 150
_i6 180
train_RD 9024
test_RD 2216
train_RA 9024
test_RA 2216
_i7 822
FrameCapture 136
_i8 643
pad_video 136
_i9 104
width 28
height 28
channels 28
num_frames 28
_i10 160
data 96
_10 28
_i11 212
print_function 56
sys 80


In [12]:
dur = 24 # sec
win = 0.2 # win len
n_step = int(dur/win)
des_w = width*n_step
des_h = height
print('num steps: '+str(n_step))
print('desired width: '+str(des_w))

num steps: 120
desired width: 15360


In [13]:
# train
videos = []
cnt = 1
for video in train_RD:
    clear_output(wait=True)
    print('Loading train video '+str(cnt)+'/'+str(len(train_RD))+ ' '+video)#,end="\r")
    vid = pad_video(FrameCapture(video))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    videos.append(vid)
    cnt += 1
del train_RD
gc.collect()
data.append(np.array(videos)/255.)

videos = []
cnt = 1
for video in train_RA:
    clear_output(wait=True)
    print('Loading train video '+str(cnt)+'/'+str(len(train_RA))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
    vid = pad_video(FrameCapture(video))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    videos.append(vid)
    cnt += 1
del train_RA
gc.collect()
data.append(np.array(videos)/255.)

# test
videos = []
cnt = 1
for video in test_RD:
    clear_output(wait=True)
    print('Loading test video '+str(cnt)+'/'+str(len(test_RD))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    vid = pad_video(FrameCapture(video))
    videos.append(vid)
    cnt += 1
del test_RD
gc.collect()
data.append(np.array(videos)/255.)

videos = []
cnt = 1
for video in test_RA:
    clear_output(wait=True)
    print('Loading test video '+str(cnt)+'/'+str(len(test_RA))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    vid = pad_video(FrameCapture(video))
    videos.append(vid)
    cnt += 1
del test_RA
gc.collect()
data.append(np.array(videos)/255.)
del videos
gc.collect()

Loading train video 1056/1056 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/22 apr sean/rangeDOA/11010002_1619121381_5.avi


MemoryError: Unable to allocate 234. GiB for an array with shape (1056, 605, 128, 128, 3) and data type float64

In [ ]:
print(data[0].shape)
print(data[1].shape)
print(data[2].shape)
print(data[3].shape)
print(data[4].shape)
print(data[5].shape)

In [ ]:
# data = [train_RD_videos, test_RD_videos, y_train, y_test, train_RA_videos, test_RA_videos]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

In [ ]:
# with open(filename, 'rb') as input:
#     x = pickle.load(input)

In [ ]:
# x[2][0].shape

In [ ]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))